<a href="https://colab.research.google.com/github/AsfandYar-Ktk/Decision-Trees/blob/main/Decision_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

In [23]:
df = pd.read_csv('/content/dataset_traffic_accident_prediction1.csv')
print(df)

    Weather   Road_Type Time_of_Day  Traffic_Density  Speed_Limit  \
0     Rainy   City Road     Morning              1.0        100.0   
1     Clear  Rural Road       Night              NaN        120.0   
2     Rainy     Highway     Evening              1.0         60.0   
3     Clear   City Road   Afternoon              2.0         60.0   
4     Rainy     Highway     Morning              1.0        195.0   
..      ...         ...         ...              ...          ...   
835   Clear     Highway       Night              2.0         30.0   
836   Rainy  Rural Road     Evening              2.0         60.0   
837   Foggy     Highway     Evening              NaN         30.0   
838   Foggy     Highway   Afternoon              2.0         60.0   
839   Clear     Highway   Afternoon              1.0         60.0   

     Number_of_Vehicles  Driver_Alcohol Accident_Severity      Road_Condition  \
0                   5.0             0.0               NaN                 Wet   
1        

# Preprocessing

For now I have removed the NaN valued rows. In future pre-process the missing values better.

In [51]:
df_copy = df.copy()

In [26]:
df_copy.Accident.value_counts()

,count
Accident,
0.0,559
1.0,239


In [36]:
df_copy['Accident'] = [0 if (x == None) else x for x in df_copy['Accident']]

Convert string values to number

In [43]:
df_copy['Weather'].value_counts()

,count
Weather,
Clear,334
Rainy,230
Foggy,107
Snowy,87
Stormy,40


In [ ]:
# prompt: in the df_copy dataframe. one hot encode the weather Feature for all of its categories.

import pandas as pd
# Assuming df_copy is already defined and loaded with data

# Perform one-hot encoding on the 'Weather' column
weather_dummies = pd.get_dummies(df_copy['Weather'], prefix='Weather')

# Concatenate the one-hot encoded columns with the original DataFrame
df_copy = pd.concat([df_copy, weather_dummies], axis=1)

# Optionally, drop the original 'Weather' column if no longer needed
df_copy = df_copy.drop('Weather', axis=1)

# Display the updated DataFrame (optional)
print(df_copy.head())


In [47]:
weather_dummies = pd.get_dummies(df_copy['Weather'], prefix='Weather')
print(weather_dummies)

     Weather_Clear  Weather_Foggy  Weather_Rainy  Weather_Snowy  \
0            False          False           True          False   
1             True          False          False          False   
2            False          False           True          False   
3             True          False          False          False   
4            False          False           True          False   
..             ...            ...            ...            ...   
835           True          False          False          False   
836          False          False           True          False   
837          False           True          False          False   
838          False           True          False          False   
839           True          False          False          False   

     Weather_Stormy  
0             False  
1             False  
2             False  
3             False  
4             False  
..              ...  
835           False  
836           False

In [52]:
df_copy = pd.concat([df_copy, weather_dummies], axis = 1)
print(df_copy.head())

  Weather   Road_Type Time_of_Day  Traffic_Density  Speed_Limit  \
0   Rainy   City Road     Morning              1.0        100.0   
1   Clear  Rural Road       Night              NaN        120.0   
2   Rainy     Highway     Evening              1.0         60.0   
3   Clear   City Road   Afternoon              2.0         60.0   
4   Rainy     Highway     Morning              1.0        195.0   

   Number_of_Vehicles  Driver_Alcohol Accident_Severity      Road_Condition  \
0                 5.0             0.0               NaN                 Wet   
1                 3.0             0.0          Moderate                 Wet   
2                 4.0             0.0               Low                 Icy   
3                 3.0             0.0               Low  Under Construction   
4                11.0             0.0               Low                 Dry   

  Vehicle_Type  Driver_Age  Driver_Experience Road_Light_Condition  Accident  \
0          Car        51.0               4

In [53]:
df_copy = df_copy.drop('Weather', axis = 1)
print(df_copy.head())

    Road_Type Time_of_Day  Traffic_Density  Speed_Limit  Number_of_Vehicles  \
0   City Road     Morning              1.0        100.0                 5.0   
1  Rural Road       Night              NaN        120.0                 3.0   
2     Highway     Evening              1.0         60.0                 4.0   
3   City Road   Afternoon              2.0         60.0                 3.0   
4     Highway     Morning              1.0        195.0                11.0   

   Driver_Alcohol Accident_Severity      Road_Condition Vehicle_Type  \
0             0.0               NaN                 Wet          Car   
1             0.0          Moderate                 Wet        Truck   
2             0.0               Low                 Icy          Car   
3             0.0               Low  Under Construction          Bus   
4             0.0               Low                 Dry          Car   

   Driver_Age  Driver_Experience Road_Light_Condition  Accident  \
0        51.0            

One hot encode Road_Type

In [55]:
df_copy['Road_Type'].value_counts()

,count
Road_Type,
Highway,402
City Road,230
Rural Road,125
Mountain Road,41


In [58]:
road_dummies = pd.get_dummies(df_copy['Road_Type'], prefix="Road")
print(road_dummies)

     Road_City Road  Road_Highway  Road_Mountain Road  Road_Rural Road
0              True         False               False            False
1             False         False               False             True
2             False          True               False            False
3              True         False               False            False
4             False          True               False            False
..              ...           ...                 ...              ...
835           False          True               False            False
836           False         False               False             True
837           False          True               False            False
838           False          True               False            False
839           False          True               False            False

[840 rows x 4 columns]


In [67]:
road_dummies = road_dummies.rename(columns={'Road_City Road': 'Road_City_Road'})

In [66]:
road_dummies = road_dummies.rename(columns={'Road_Mountain Road': 'Road_Mountain_Road'})

In [64]:
road_dummies = road_dummies.rename(columns={'Road_Rural Road': 'Road_Rural_Road'})

In [68]:
road_dummies

,Road_City_Road,Road_Highway,Road_Mountain_Road,Road_Rural_Road
0,True,False,False,False
1,False,False,False,True
2,False,True,False,False
3,True,False,False,False
4,False,True,False,False
...,...,...,...,...
835,False,True,False,False
836,False,False,False,True
837,False,True,False,False
838,False,True,False,False


In [71]:
df_copy = pd.concat([df_copy, road_dummies], axis=1)
df_copy

,Road_Type,Time_of_Day,Traffic_Density,Speed_Limit,Number_of_Vehicles,Driver_Alcohol,Accident_Severity,Road_Condition,Vehicle_Type,Driver_Age,...,Road_Mountain_Road,Road_Rural_Road,Road_City_Road,Road_Highway,Road_Mountain_Road,Road_Rural_Road,Road_City_Road,Road_Highway,Road_Mountain_Road,Road_Rural_Road
0,City Road,Morning,1.0,100.0,5.0,0.0,NaN,Wet,Car,51.0,...,False,False,True,False,False,False,True,False,False,False
1,Rural Road,Night,NaN,120.0,3.0,0.0,Moderate,Wet,Truck,49.0,...,False,True,False,False,False,True,False,False,False,True
2,Highway,Evening,1.0,60.0,4.0,0.0,Low,Icy,Car,54.0,...,False,False,False,True,False,False,False,True,False,False
3,City Road,Afternoon,2.0,60.0,3.0,0.0,Low,Under Construction,Bus,34.0,...,False,False,True,False,False,False,True,False,False,False
4,Highway,Morning,1.0,195.0,11.0,0.0,Low,Dry,Car,62.0,...,False,False,False,True,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,Highway,Night,2.0,30.0,4.0,0.0,Low,Dry,Car,23.0,...,False,False,False,True,False,False,False,True,False,False
836,Rural Road,Evening,2.0,60.0,4.0,0.0,Low,Dry,Motorcycle,52.0,...,False,True,False,False,False,True,False,False,False,True
837,Highway,Evening,NaN,30.0,4.0,1.0,High,Dry,Car,NaN,...,False,False,False,True,False,False,False,True,False,False
838,Highway,Afternoon,2.0,60.0,3.0,0.0,Low,Dry,Car,25.0,...,False,False,False,True,False,False,False,True,False,False


In [72]:
df_copy = df_copy.drop('Road_Type', axis=1)
df_copy

,Time_of_Day,Traffic_Density,Speed_Limit,Number_of_Vehicles,Driver_Alcohol,Accident_Severity,Road_Condition,Vehicle_Type,Driver_Age,Driver_Experience,...,Road_Mountain_Road,Road_Rural_Road,Road_City_Road,Road_Highway,Road_Mountain_Road,Road_Rural_Road,Road_City_Road,Road_Highway,Road_Mountain_Road,Road_Rural_Road
0,Morning,1.0,100.0,5.0,0.0,NaN,Wet,Car,51.0,48.0,...,False,False,True,False,False,False,True,False,False,False
1,Night,NaN,120.0,3.0,0.0,Moderate,Wet,Truck,49.0,43.0,...,False,True,False,False,False,True,False,False,False,True
2,Evening,1.0,60.0,4.0,0.0,Low,Icy,Car,54.0,52.0,...,False,False,False,True,False,False,False,True,False,False
3,Afternoon,2.0,60.0,3.0,0.0,Low,Under Construction,Bus,34.0,31.0,...,False,False,True,False,False,False,True,False,False,False
4,Morning,1.0,195.0,11.0,0.0,Low,Dry,Car,62.0,55.0,...,False,False,False,True,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,Night,2.0,30.0,4.0,0.0,Low,Dry,Car,23.0,15.0,...,False,False,False,True,False,False,False,True,False,False
836,Evening,2.0,60.0,4.0,0.0,Low,Dry,Motorcycle,52.0,46.0,...,False,True,False,False,False,True,False,False,False,True
837,Evening,NaN,30.0,4.0,1.0,High,Dry,Car,NaN,34.0,...,False,False,False,True,False,False,False,True,False,False
838,Afternoon,2.0,60.0,3.0,0.0,Low,Dry,Car,25.0,19.0,...,False,False,False,True,False,False,False,True,False,False


One hot encode Time_of_day

In [73]:
df_copy['Time_of_Day'].value_counts()

,count
Time_of_Day,
Afternoon,272
Evening,218
Morning,201
Night,107


In [74]:
time_dummies = pd.get_dummies(df_copy['Time_of_Day'], prefix='Time')
print(time_dummies)

     Time_Afternoon  Time_Evening  Time_Morning  Time_Night
0             False         False          True       False
1             False         False         False        True
2             False          True         False       False
3              True         False         False       False
4             False         False          True       False
..              ...           ...           ...         ...
835           False         False         False        True
836           False          True         False       False
837           False          True         False       False
838            True         False         False       False
839            True         False         False       False

[840 rows x 4 columns]


In [75]:
df_copy = pd.concat([df_copy, time_dummies], axis=1)

In [76]:
df_copy

,Time_of_Day,Traffic_Density,Speed_Limit,Number_of_Vehicles,Driver_Alcohol,Accident_Severity,Road_Condition,Vehicle_Type,Driver_Age,Driver_Experience,...,Road_Mountain_Road,Road_Rural_Road,Road_City_Road,Road_Highway,Road_Mountain_Road,Road_Rural_Road,Time_Afternoon,Time_Evening,Time_Morning,Time_Night
0,Morning,1.0,100.0,5.0,0.0,NaN,Wet,Car,51.0,48.0,...,False,False,True,False,False,False,False,False,True,False
1,Night,NaN,120.0,3.0,0.0,Moderate,Wet,Truck,49.0,43.0,...,False,True,False,False,False,True,False,False,False,True
2,Evening,1.0,60.0,4.0,0.0,Low,Icy,Car,54.0,52.0,...,False,False,False,True,False,False,False,True,False,False
3,Afternoon,2.0,60.0,3.0,0.0,Low,Under Construction,Bus,34.0,31.0,...,False,False,True,False,False,False,True,False,False,False
4,Morning,1.0,195.0,11.0,0.0,Low,Dry,Car,62.0,55.0,...,False,False,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,Night,2.0,30.0,4.0,0.0,Low,Dry,Car,23.0,15.0,...,False,False,False,True,False,False,False,False,False,True
836,Evening,2.0,60.0,4.0,0.0,Low,Dry,Motorcycle,52.0,46.0,...,False,True,False,False,False,True,False,True,False,False
837,Evening,NaN,30.0,4.0,1.0,High,Dry,Car,NaN,34.0,...,False,False,False,True,False,False,False,True,False,False
838,Afternoon,2.0,60.0,3.0,0.0,Low,Dry,Car,25.0,19.0,...,False,False,False,True,False,False,True,False,False,False


In [78]:
df_copy = df_copy.drop('Time_of_Day', axis=1)
print(df_copy.head())

   Traffic_Density  Speed_Limit  Number_of_Vehicles  Driver_Alcohol  \
0              1.0        100.0                 5.0             0.0   
1              NaN        120.0                 3.0             0.0   
2              1.0         60.0                 4.0             0.0   
3              2.0         60.0                 3.0             0.0   
4              1.0        195.0                11.0             0.0   

  Accident_Severity      Road_Condition Vehicle_Type  Driver_Age  \
0               NaN                 Wet          Car        51.0   
1          Moderate                 Wet        Truck        49.0   
2               Low                 Icy          Car        54.0   
3               Low  Under Construction          Bus        34.0   
4               Low                 Dry          Car        62.0   

   Driver_Experience Road_Light_Condition  ...  Road_Mountain_Road  \
0               48.0     Artificial Light  ...               False   
1               43.0    

In [83]:
df_copy['Speed_Limit'].isna().sum()

42

In [90]:
df_copy_2 = df_copy.copy()
df_copy_2 = df_copy_2.Speed_Limit.fillna(60)
print(df_copy_2)

0      100.0
1      120.0
2       60.0
3       60.0
4      195.0
       ...  
835     30.0
836     60.0
837     30.0
838     60.0
839     60.0
Name: Speed_Limit, Length: 840, dtype: float64


# Decision Trees From Scratch